# Homework: Agents

## Preparation

In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

## Q1. Define function description

In [2]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get the current temperature for a given city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city to get the weather for."
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

## Q2. Adding another tool

In [3]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [6]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Set the temperature for a given city in the weather database.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city to set the weather for."
            },
            "temp": {
                "type": "number",
                "description": "Temperature value to set for the city."
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

## Q3. Install FastMCP

In [4]:
! uv pip install fastmcp

Using Python 3.10.18 environment at: C:\Users\tinyu\anaconda3\envs\agents
Resolved 46 packages in 5.51s
   Building pyperclip==1.9.0
      Built pyperclip==1.9.0
Prepared 26 packages in 1.32s
Uninstalled 1 package in 1.73s
Installed 42 packages in 2.06s
 + annotated-types==0.7.0
 + anyio==4.9.0
 + attrs==25.3.0
 + authlib==1.6.0
 + certifi==2025.7.9
 + cffi==1.17.1
 + click==8.2.1
 + cryptography==45.0.5
 + cyclopts==3.22.2
 + dnspython==2.7.0
 + docstring-parser==0.16
 + docutils==0.21.2
 + email-validator==2.2.0
 + fastmcp==2.10.5
 + h11==0.16.0
 + httpcore==1.0.9
 + httpx==0.28.1
 + httpx-sse==0.4.1
 + idna==3.10
 + jsonschema==4.24.0
 + jsonschema-specifications==2025.4.1
 + markdown-it-py==3.0.0
 + mcp==1.11.0
 + mdurl==0.1.2
 + openapi-pydantic==0.5.1
 + pycparser==2.22
 + pydantic==2.11.7
 + pydantic-core==2.33.2
 + pydantic-settings==2.10.1
 + pyperclip==1.9.0
 + python-dotenv==1.1.1
 + python-multipart==0.0.20
 - pywin32==307
 + pywin32==310
 + referencing==0.36.2
 + rich==14.

In [5]:
!conda list fastmcp

# packages in environment at C:\Users\tinyu\anaconda3\envs\agents:
#
# Name                    Version                   Build  Channel
fastmcp                   2.10.5                   pypi_0    pypi


## Q4. Simple MCP Server

# weather_server.py
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

if __name__ == "__main__":
    mcp.run()

## Q5. Protocol

In [28]:
import requests
import json

# Make sure your server is running with: python weather_server.py
url = "http://127.0.0.1:8001/mcp"
headers = {"Content-Type": "application/json"}

# 1. Initialize
init_payload = {
    "jsonrpc": "2.0", 
    "id": 1, 
    "method": "initialize", 
    "params": {
        "protocolVersion": "2024-11-05", 
        "capabilities": {"roots": {"listChanged": True}, "sampling": {}}, 
        "clientInfo": {"name": "test-client", "version": "1.0.0"}
    }
}

response = requests.post(url, json=init_payload, headers=headers)
print("Initialize:", response.json())

# 2. Confirm initialization
confirm_payload = {"jsonrpc": "2.0", "method": "notifications/initialized"}
requests.post(url, json=confirm_payload, headers=headers)

# 3. List tools
list_payload = {"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
response = requests.post(url, json=list_payload, headers=headers)
print("Tools:", response.json())

# 4. Call weather for Berlin
weather_payload = {
    "jsonrpc": "2.0",
    "id": 3,
    "method": "tools/call",
    "params": {
        "name": "get_weather",
        "arguments": {"city": "berlin"}
    }
}

response = requests.post(url, json=weather_payload, headers=headers)
print("Weather result:", response.json())

Initialize: {'jsonrpc': '2.0', 'id': 'server-error', 'error': {'code': -32600, 'message': 'Not Acceptable: Client must accept both application/json and text/event-stream'}}
Tools: {'jsonrpc': '2.0', 'id': 'server-error', 'error': {'code': -32600, 'message': 'Not Acceptable: Client must accept both application/json and text/event-stream'}}
Weather result: {'jsonrpc': '2.0', 'id': 'server-error', 'error': {'code': -32600, 'message': 'Not Acceptable: Client must accept both application/json and text/event-stream'}}


## Q6. Client

In [29]:
import weather_server

In [30]:
from fastmcp import Client

In [33]:
async def main():
    async with Client(weather_server.mcp) as mcp_client:
        # Get list of available tools
        tools = await mcp_client.list_tools()
        print("Available tools:", tools)
        
        # Call get_weather for Berlin
        result = await mcp_client.call_tool("get_weather", {"city": "berlin"})
        print("Weather result:", result)
        
        return tools

# Run the async function
tools_result = await main()

Available tools: [Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'tem

## Using tools from the MCP server (optional)

In [38]:
import mcp_client

our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Weather Server 🌦️', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully


In [39]:
our_mcp_client.get_tools()
our_mcp_client.call_tool('get_weather', {'city': 'Berlin'})

Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'Berlin'}


{'content': [{'type': 'text', 'text': '20.0'}],
 'structuredContent': {'result': 20.0},
 'isError': False}

In [47]:
!uv pip install mistralai

Using Python 3.10.18 environment at: C:\Users\tinyu\anaconda3\envs\agents
Audited 1 package in 25ms


In [4]:
!uv pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Using Python 3.10.18 environment at: C:\Users\tinyu\anaconda3\envs\agents
Audited 1 package in 21ms
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run script troubleshoot

Jupyter command `jupyter-nbextension` not found.


In [6]:
import os

In [9]:
import mcp_client
import chat_assistant_ui
import os

# Your Mistral API key is already set

# Create MCP client
our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

mcp_tools = mcp_client.MCPTools(mcp_client=our_mcp_client)

developer_prompt = """
You help users find out the weather in their cities. 
If they didn't specify a city, ask them. Make sure we always use a city.
You can also set weather for cities if users request it.
""".strip()

# Create and start the enhanced chat assistant
chat = chat_assistant_ui.ChatAssistantUI(
    tools=mcp_tools,
    developer_prompt=developer_prompt,
    api_key=os.getenv("MISTRAL_API_KEY")  # or pass your API key directly
)

chat.start()

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Weather Server 🌦️', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully
